##### Chernobyl
https://catalogue.ceh.ac.uk/documents/a657ffc3-8f62-458f-bcb7-30e116807174

In [21]:
import requests
import cv2
import pandas as pd
import numpy as np

# from ..utils.detector import Detector
%run ../utils/detector.py

In [9]:
detector = Detector()
df = pd.read_csv('CHAR_Nov_2020_Mar_2021_Camera_Trap_Image_Catalogue.csv')
df.Species_Common_Name.unique()

Using deepfaune-yolov8s_960 with weights at ../deepfaune/models/deepfaune-yolov8s_960.pt, in resolution 960x960


array(['Service', 'Red deer', 'Eurasian elk', 'Nothing', 'Grey wolf', 'Brown hare', 'Unidentifiable', 'Vehicles', 'Red fox', 'Roe deer', 'Wild boar', 'Tawny Owl', 'Marten sp. ', 'Eurasian lynx', "Przewalski's horse", 'People ', 'People', 'European badger', 'Spotted nutcracker', 'Red squirrel', 'Raccoon dog',
       'Unidentifiable bird', 'Great tit', 'European otter', 'Bullfinch'], dtype=object)

In [1]:
species_names = ['badger', 'squirrel', 'raccoon dog', 'bird', 'red deer', 'wolf', 'fox', 'roe deer', 'hare', 'wild boar', 'moose', 'otter']
data_species_names = ['European badger', 'Red squirrel', 'Raccoon dog', 'Unidentifiable bird', 'Red deer', 'Grey wolf', 'Red fox', 'Roe deer', 'Brown hare', 'Wild boar', 'Eurasian elk', 'European otter']
species_names = ['wolf', 'fox', 'roe deer', 'hare', 'wild boar', 'moose', 'otter']
data_species_names = ['Grey wolf', 'Red fox', 'Roe deer', 'Brown hare', 'Wild boar', 'Eurasian elk', 'European otter']

species = pd.DataFrame({'species_name': species_names, 'data_species_name': data_species_names})

NameError: name 'pd' is not defined

In [ ]:
for _, row_species in species.iterrows():
    species = row_species['species_name']
    data_species = row_species['data_species_name']
    observations = df[df.Species_Common_Name == data_species]
    print(species)
    i: int = 1

    for _, row in observations.iterrows():
        if i > 1000:
            break

        subdir1 = row.Image_Location_Folder_Name[:11] + '_' + row.Image_Location_Folder_Name[11] + '/'
        subdir2 = row.Image_Location_Folder_Name + '/'
        subdir3 = data_species.replace(' ', '') + '/'
        file = row.Image_Filename
        url = 'https://catalogue.ceh.ac.uk/datastore/eidchub/a657ffc3-8f62-458f-bcb7-30e116807174/' + subdir1 + subdir2 + subdir3 + file

        # download and crop
        response = requests.get(url)
        try:
            response.raise_for_status()
        except:
            print(f'File {url} missing')
            continue

        # Convert bytes → NumPy array → OpenCV image
        image_arr = np.frombuffer(response.content, np.uint8)
        image = cv2.imdecode(image_arr, cv2.IMREAD_COLOR)
        cropped_img, category, box, count, humanboxes = detector.bestBoxDetection(image)

        # save to species/pictures_cropped
        if cropped_img and category == 1:
            cropped_img.save(f"{species}/pictures_cropped/ch_{i}.jpg", format="JPEG")
            i += 1

badger
squirrel
raccoon dog
bird
red deer
